In [5]:
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import seaborn as sns
import nltk
from collections import Counter
from yellowbrick.text import PosTagVisualizer
import contractions
import os

In [92]:
df = pd.read_csv("tweets_dataset.csv", encoding = "latin-1")
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [93]:
df.columns = ["Sentiment Level", "Index", "Date", "No_Query", "Username", "Tweet"]
df.drop("No_Query", inplace = True, axis = 1)

In [94]:
df

,Sentiment Level,Index,Date,Username,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [95]:
df_text = df["Tweet"]
df_class = df["Sentiment Level"]

(1599999,)


In [96]:
df = pd.concat([df_text, df_class], axis = 1)
df_negative = df.iloc[0:40000]
df_positive = df.loc[df["Sentiment Level"] == 4]
df_positive = df_positive.iloc[0:40000]

In [213]:
df_concat = pd.concat([df_negative, df_positive], axis = 0)

In [214]:
df_concat

,Tweet,Sentiment Level
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
...,...,...
839994,Oficially done with drivers ed stuff. Now I ju...,4
839995,@jimhunt Thank YOU for sharing and caring! Gl...,4
839996,@buberzionist ok objection sustained,4
839997,is twittin' while my baby girl sleeps,4


In [216]:
#removing punctuation 
df_concat["Tweet"] = df_concat["Tweet"].str.replace(r'[^\w\s]+', '', regex = True)

C:\Users\CHEN KANG\AppData\Local\Temp\ipykernel_10464\2156519555.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_concat["Tweet"] = df_concat["Tweet"].str.replace(r'[^\w\s]+', '')


In [215]:
#removing usernames that starts with @
df_concat["Tweet"] = df_concat["Tweet"].str.replace("@[^\s]+", "", regex = True)



In [217]:
#removing emoji 
emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
    "]+", re.UNICODE)

df_concat["Tweet"] = df_concat["Tweet"].apply(lambda x : re.sub(emoj, "", x))
df_concat["Tweet"] = df_concat["Tweet"].apply(lambda x : x.encode("ascii", "ignore").decode())

In [218]:
#contractions
df_concat['Tweet'] = df_concat['Tweet'].apply(lambda x: [contractions.fix(word) for word in x.split()])
df_concat["Tweet"] = [' '.join(map(str,l)) for l in df_concat["Tweet"]]


In [219]:
df_concat

,Tweet,Sentiment Level
0,is upset that he cannot update his Facebook by...,0
1,I dived many times for the ball Managed to sav...,0
2,my whole body feels itchy and like its on fire,0
3,no its not behaving at all i am mad why am i h...,0
4,not the whole crew,0
...,...,...
839994,Oficially done with drivers ed stuff Now I jus...,4
839995,Thank YOU for sharing and caring Glad that you...,4
839996,ok objection sustained,4
839997,is twittin while my baby girl sleeps,4


In [209]:
max = 0;

for rows in df_concat["Tweet"]: 
    if len(rows) > max:
        max = len(rows)
        
print(max)

158


In [220]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

#stopwords
df_concat["Tweet"] = df_concat["Tweet"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)])) 

In [221]:
df_concat

,Tweet,Sentiment Level
0,upset cannot update Facebook texting might cry...,0
1,I dived many times ball Managed save 50 The re...,0
2,whole body feels itchy like fire,0
3,behaving mad I cannot see,0
4,whole crew,0
...,...,...
839994,Oficially done drivers ed stuff Now I wait Jun...,4
839995,Thank YOU sharing caring Glad like quotes,4
839996,ok objection sustained,4
839997,twittin baby girl sleeps,4


In [223]:
#tokenize the words
from nltk.tokenize import word_tokenize
df_concat["Tweet_tokenize"] = df_concat["Tweet"].apply(word_tokenize)
df_concat["Tweet"] = ["".join(map(str,l)) for l in df_concat["Tweet"]]
df_concat.head() 

,Tweet,Sentiment Level,Tweet_tokenize
0,upset cannot update Facebook texting might cry...,0,"[upset, can, not, update, Facebook, texting, m..."
1,I dived many times ball Managed save 50 The re...,0,"[I, dived, many, times, ball, Managed, save, 5..."
2,whole body feels itchy like fire,0,"[whole, body, feels, itchy, like, fire]"
3,behaving mad I cannot see,0,"[behaving, mad, I, can, not, see]"
4,whole crew,0,"[whole, crew]"


In [224]:
df_concat = df_concat.sample(frac = 1)

In [229]:
df_concat

,Tweet,Sentiment Level,Tweet_tokenize
815320,stellas brunch roomies amp birthday girl,4,"[stellas, brunch, roomies, amp, birthday, girl]"
817092,MORNING PPLS,4,"[MORNING, PPLS]"
839177,Please give shoutout Mary South Carolina It to...,4,"[Please, give, shoutout, Mary, South, Carolina..."
19560,cats longest time One meanest cats ever,0,"[cats, longest, time, One, meanest, cats, ever]"
803763,hey thanks follow,4,"[hey, thanks, follow]"
...,...,...,...
809755,A beautiful morning hugs,4,"[A, beautiful, morning, hugs]"
6400,Stressing methodolgy section mlearning report,0,"[Stressing, methodolgy, section, mlearning, re..."
20874,I sorry If I warp Id I wish I comfort right,0,"[I, sorry, If, I, warp, Id, I, wish, I, comfor..."
18764,mad coachella tomorow nerrrr mind conneticut t...,0,"[mad, coachella, tomorow, nerrrr, mind, connet..."


In [233]:
df_concat.to_csv("L:\\ML-Assignment\\training_data.csv", index = False, columns = ["Tweet_tokenize", "Sentiment Level"])